In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
import cv2
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

def get_face_from_image(img):
    gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face.detectMultiScale(gray, 1.2, 5)
    return (True, faces[0]) if len(faces)>0 else (False, None)

In [3]:
def confusion_matrix_plot(y_true, y_pred, labels=['deceptive', 'truthful'], title='Confusion Matrix'):
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    fmt = 'd'
    thresh = cm.max()/2  
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if (cm[i, j] > thresh) else "black")

    plt.ylabel('True label')
    plt.xlabel('Predited label')
    plt.tight_layout()
    plt.show()

In [4]:
video_inf_path = "D:/Miscellaneous/J-comp/MLA/RLT/Annotation/Labels.csv"

video_data = pd.read_csv(video_inf_path)
print(video_data)
video_filenames = video_data['id'].tolist()
labels = [1 if x == 'truthful' else 0 for x in video_data['class'].tolist()]
video_dict = dict(zip(video_filenames, labels))
print(video_dict)

                      id  OtherGestures  Smile  Laugh  Scowl  \
0      trial_lie_001.mp4              1      0      0      0   
1      trial_lie_002.mp4              1      0      0      0   
2      trial_lie_003.mp4              1      0      0      0   
3      trial_lie_004.mp4              1      0      0      0   
4      trial_lie_005.mp4              1      0      0      0   
..                   ...            ...    ...    ...    ...   
116  trial_truth_056.mp4              1      0      0      0   
117  trial_truth_057.mp4              1      0      0      0   
118  trial_truth_058.mp4              1      0      0      0   
119  trial_truth_059.mp4              0      0      0      1   
120  trial_truth_060.mp4              0      1      0      0   

     otherEyebrowMovement  Frown  Raise  OtherEyeMovements  Close-R  ...  \
0                       1      0      0                  1        0  ...   
1                       0      1      0                  1        0  ...   
2  

In [5]:
dataset_path = "D:/Miscellaneous/J-comp/MLA/RLT/"
video_path = "D:/Miscellaneous/J-comp/MLA/RLT/Video_chunks/"
whole_video_path = "D:/Miscellaneous/J-comp/MLA/RLT/Whole_Videos/" 

In [9]:
def preprocess_image(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray_img, (48, 48)), -1), 0)
    cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
    return cropped_img

def read_video(filename, max_frames=300):
    cam = cv2.VideoCapture(video_path + filename)
    current_frame = 0
    xdata = np.zeros((max_frames, 48*48))

    while(True):
        ret, frame_image = cam.read()
        
        if current_frame >= max_frames:
            break

        if ret:
            (detected, rect) = get_face_from_image(frame_image)
            x = np.zeros(48*48)

            if detected:
                (x, y, w, h) = rect
                cropped_face = frame_image[y:y+h, x:x+w]
                cropped_img = preprocess_image(cropped_face)
                x = cropped_img.flatten()
            
            xdata[current_frame, :] = x
            current_frame+=1

        else:
            break
    
    cam.release()
    cv2.destroyAllWindows()
    return xdata

def saveData(X, y):
    X = np.asarray(X, dtype=object)
    y = np.asarray(y, dtype=object)
    np.save(dataset_path + 'f_files/' + 'faces_dataX', X)
    np.save(dataset_path + 'f_files/' + 'faces_labels', y)
    print(f"Saving X and y of shape {X.shape}, {y.shape}, respectively.")

X = []
y = []

count = 0
for (k, v) in video_dict.items():
    count+=1
    print(k, v)
    xi = read_video(k)
    X.append(xi)
    y.append(v)
    if count % 5 == 0:
        saveData(X, y)

saveData(X, y)        

trial_lie_001.mp4 0


KeyboardInterrupt: 

In [6]:
X = np.load(dataset_path + "f_files/faces_dataX.npy", allow_pickle=True)
y = np.load(dataset_path + "f_files/faces_labels.npy", allow_pickle=True)
print('Loaded data', X.shape, y.shape)

Loaded data (121, 300, 2304) (121,)


In [7]:
X = np.reshape(X,(X.shape[0], X.shape[1], 48, 48))
X = np.expand_dims(X, -1)
X.shape

(121, 300, 48, 48, 1)

# GRU Model

In [8]:
from keras import initializers
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Masking, GRU
from keras.layers import Conv1D, Conv2D, Conv3D, MaxPooling1D, MaxPooling2D, MaxPooling3D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7983767418283203032
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2254700544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10007922531149613348
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [11]:
tf.test.is_built_with_cuda()

True

In [13]:
loaded_model = load_model("C:/Users/L3NOVO/AppData/Roaming/jupyter/kernels/mlajcomp/fer_optimal.h5")

layer_names = [layer.name for layer in loaded_model.layers]
layer_outputs = [layer.output for layer in loaded_model.layers]
print(f"layer_names total: {len(layer_names)}")

X = np.array(X, dtype=float)

X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

encoding_model = Model(inputs = loaded_model.inputs, outputs = layer_outputs[23])

X_cnn = np.zeros((X.shape[0], X.shape[1], 512))
i = 0

for (xi, yi) in zip(X, y):
    predictedX = encoding_model.predict(xi)
    X_cnn[i] = predictedX
    i+=1

X_train, X_test, y_train, y_test = train_test_split(X_cnn, y, test_size=0.1, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=7)

np.save(dataset_path + "mod_files/" + "modXtest", X_test)
np.save(dataset_path + "mod_files/" + "modytest", y_test)

layer_names total: 25
10/10 [==============================] - 0s 9ms/step


In [14]:
print(X_train.shape, y_train.shape)
m = X_train.shape[0]
Tx = X_train.shape[1]
dims = X_train.shape[2]

(97, 300, 512) (97,)


In [15]:
def create_gru_model(input_shape):
    X_input = Input(shape=input_shape)
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = MaxPooling1D(pool_size=30, strides=15, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = Dropout(rate=0.8)(X)
    
    X = GRU(units=256, return_sequences=True)(X)
    X = Dropout(rate=0.8)(X)
    #X = BatchNormalization()(X)

    # X = GRU(units=256, return_sequences=True)(X)
    # X = Dropout(rate=0.8)(X)
    # X = BatchNormalization()(X)

    X = GRU(units=256, return_sequences=True)(X)
    X = Dropout(rate=0.8)(X)
    X = BatchNormalization()(X)

    X = Dense(1, activation="sigmoid")(X)
    X = Flatten()(X)

    model = Model(inputs=X_input, outputs=X)

    return model

gru_model = create_gru_model(input_shape=(Tx, dims))
gru_model.summary()    

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 512)]        0         
                                                                 
 conv1d (Conv1D)             (None, 72, 196)           1505476   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 5, 196)           0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 5, 196)           784       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 5, 196)            0         
                                                                 
 dropout (Dropout)           (None, 5, 196)            0   

In [16]:
gru_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01, epsilon=1e-7), metrics=['accuracy'])

X_train = np.asarray(X_train).astype(np.int)
y_train=np.asarray(y_train).astype(np.int)
X_valid=np.asarray(X_valid).astype(np.int)
y_valid=np.asarray(y_valid).astype(np.int)

gru_model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=150,
    verbose=1,
    validation_data=(X_valid, y_valid),
    shuffle=True
)

_, accuracy = gru_model.evaluate(X_valid, y_valid)
acc = '%.2f' % (accuracy*100)
print(f'Accuracy: {acc}%')

gru_model.save("gru_model.h5")
gru_model.save_weights("gru_model_weights.h5")
print("Saved gru_model to disk")

Epoch 1/150
1/1 [==============================] - 11s 11s/step - loss: 0.8531 - accuracy: 0.1856 - val_loss: 0.5609 - val_accuracy: 0.8182
Epoch 2/150
1/1 [==============================] - 0s 156ms/step - loss: 0.8843 - accuracy: 0.1856 - val_loss: 0.5317 - val_accuracy: 0.8182
Epoch 3/150
1/1 [==============================] - 0s 168ms/step - loss: 0.7991 - accuracy: 0.1959 - val_loss: 0.5094 - val_accuracy: 0.8182
Epoch 4/150
1/1 [==============================] - 0s 137ms/step - loss: 0.8259 - accuracy: 0.2680 - val_loss: 0.5040 - val_accuracy: 0.8182
Epoch 5/150
1/1 [==============================] - 0s 164ms/step - loss: 0.8164 - accuracy: 0.2577 - val_loss: 0.5075 - val_accuracy: 0.8182
Epoch 6/150
1/1 [==============================] - 0s 143ms/step - loss: 0.8366 - accuracy: 0.2165 - val_loss: 0.5084 - val_accuracy: 0.8182
Epoch 7/150
1/1 [==============================] - 0s 152ms/step - loss: 0.7784 - accuracy: 0.1753 - val_loss: 0.5095 - val_accuracy: 0.8182
Epoch 8/150
1/

In [17]:
y_train

array([1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0])

In [18]:
gru_model.predict(X_train)

4/4 [==============================] - 2s 40ms/step


array([[9.67769682e-01, 9.99444783e-01, 9.99964118e-01, 9.99991894e-01,
        9.99996781e-01],
       [1.48760516e-03, 9.07743561e-06, 5.86755164e-07, 1.37102617e-07,
        6.92975846e-08],
       [1.13580050e-03, 7.50711297e-06, 5.40555561e-07, 1.41185978e-07,
        8.57501732e-08],
       [9.59256709e-01, 9.99050200e-01, 9.99955416e-01, 9.99992013e-01,
        9.99996185e-01],
       [9.83258843e-01, 9.99823630e-01, 9.99995112e-01, 9.99999523e-01,
        9.99999881e-01],
       [9.63202238e-01, 9.99214649e-01, 9.99954462e-01, 9.99989152e-01,
        9.99992132e-01],
       [7.17637420e-04, 5.28736336e-06, 4.39200875e-07, 1.71790802e-07,
        1.38955215e-07],
       [4.51980717e-03, 3.84005871e-05, 1.60765774e-06, 2.76027748e-07,
        1.20759552e-07],
       [1.25847710e-02, 6.55307231e-05, 2.59866056e-06, 4.83915926e-07,
        1.94566837e-07],
       [9.96229589e-01, 9.99975562e-01, 9.99998689e-01, 9.99999762e-01,
        9.99999881e-01],
       [9.93217647e-01, 9.9994

In [20]:
X_train_res = gru_model.predict(X_train)

truth_res = []
lie_res = []
for xi, yi in zip(X_train_res, y_train):
  if yi == 1:
    truth_res.append(xi)
  else:
    lie_res.append(xi)
print(f"There are {len(truth_res)} truths and {len(lie_res)} lies.")

average_truth_res = np.mean(truth_res, axis=0)
print(f"Average truth results: {average_truth_res}")
average_lie_res = np.mean(lie_res, axis=0)
print(f"Average lie results: {average_lie_res}")

database={}
database['truth'] = average_truth_res
database['lie'] = average_lie_res

4/4 [==============================] - 0s 70ms/step
There are 51 truths and 46 lies.
Average truth results: [0.9810392  0.9990345  0.9999308  0.99999464 0.99999875]
Average lie results: [8.5480148e-03 1.3544103e-03 4.2632969e-06 4.2474946e-07 2.3889504e-07]


In [21]:
X_test_res = gru_model.predict(X_test)
print(f"X_test.shape={X_test.shape}")

def verify(res, gru_model, database):
    dist_truth = np.linalg.norm(res - database["truth"])
    dist_lie = np.linalg.norm(res - database["lie"])
    print(f"dist_truth={dist_truth}, dist_lie={dist_lie}")
    return 1 if dist_truth < dist_lie else 0

count = 0
y_pred = []

for res, y in zip(X_test_res, y_test):
    y_predict = verify(res, gru_model, database)
    y_pred.append(y_predict)
    if y_predict == y:
        count += 1
    else:
        print("y_predict: ", y_predict, "y: ", y)

accuracy = count/X_test_res.shape[0]
acc_str = '%.2f' % (accuracy*100)
print('Validation accuracy: ' + acc_str + '%')        

1/1 [==============================] - 1s 1s/step
X_test.shape=(13, 300, 512)
dist_truth=0.018212424591183662, dist_lie=2.2313032150268555
dist_truth=1.4483065605163574, dist_lie=0.8385909199714661
dist_truth=0.012135105207562447, dist_lie=2.228574752807617
y_predict:  1 y:  0
dist_truth=2.2123146057128906, dist_lie=0.025451812893152237
y_predict:  0 y:  1
dist_truth=0.01745789870619774, dist_lie=2.2309646606445312
dist_truth=1.7026264667510986, dist_lie=1.3191823959350586
dist_truth=0.015915647149086, dist_lie=2.230267286300659
y_predict:  1 y:  0
dist_truth=0.11793786287307739, dist_lie=2.1705970764160156
dist_truth=0.22634011507034302, dist_lie=2.1193361282348633
y_predict:  1 y:  0
dist_truth=0.008574393577873707, dist_lie=2.2270028591156006
dist_truth=2.2176380157470703, dist_lie=0.0131016094237566
y_predict:  0 y:  1
dist_truth=1.042024850845337, dist_lie=1.6041048765182495
dist_truth=2.226774215698242, dist_lie=0.007744119502604008
Validation accuracy: 61.54%
